In [ ]:
DATA_DIR = 'data/train/'
window_size = 20
EPOCHS = 150
batch_size=512
BUFFER_SIZE=40000
acquisition_cost = 50000000
cost_reactive = acquisition_cost/3
cost_predictive = cost_reactive/3

try:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Thesis/code/
except:
  colab = False

In [ ]:
import pandas as pd
import numpy as np
import os
import random 
from emp.losses.prmc.xgboost import XGBObjectiveFunction, XGBMSE, XGBPseudoHuberLoss
from emp.metrics.maintenance import calculate_PRMC
import xgboost as xgb
from xgboost import XGBRegressor,XGBRFRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import ParameterGrid

import json

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from packages.hypopt.model_selection import _compute_score


from preprocessing.keras import preprocess_test_windowed_UL, preprocess_train_windowed_UL

In [ ]:
def predict_test_set(model, test_x):
    """
    Simulate the model on the testset. 

    @test_x: list of numpy arrays representing the observed sensors throughout time
    """
    predicted_RUL_list = []
    for machine in test_x:
        if 'best_iteration' in dir(model):
          # pred_RUL = np.exp(model.predict(machine,iteration_range=(0,model.best_iteration)))
          pred_RUL = model.predict(machine,iteration_range=(0,model.best_iteration))
          predicted_RUL_list.append(pred_RUL)
        else:
          # pred_RUL = np.exp(model.predict(machine))
          pred_RUL = model.predict(machine)
          predicted_RUL_list.append(pred_RUL) 
    return predicted_RUL_list

def unravel_labels_testset(preds):
  """
  Unravel labels into a np.array 

  Args:
      preds (list[np.array]): list of numpy arrays. Each numpy array corresponds to a machine and every value in such array corresponds to 
                              a prediction for a given moment in time for that specific machine

  """

  out = np.zeros((len(preds),max(list(map(lambda x: len(x),preds)))))
  for row, machine in enumerate(preds):
      out[row, 0:len(machine)] = np.ravel(machine)

  return out

In [ ]:
def looper(files,acquisition_cost,cost_reactive,cost_predictive,MODEL_NAME,shared = False,params={
    "verbosity":1,
    'max_depth':6,
    'disable_default_eval_metric': 1,
    'min_child_weight':1
}, epochs = 150,objective=False):
  files=files.merge(files.groupby('unit_ID').max('cycles')['cycles'].rename('UL'),left_on='unit_ID',right_index=True)
  files['RUL'] = files['UL']-files['cycles'] #create 'RUL-variable' for every row

  test_ids = random.sample(list(set(files.unit_ID)),round(len(set(files.unit_ID))*0.1))

  test_data = pd.DataFrame()
  train_data = files

  for test_id in test_ids:
      
      test_data = pd.concat([test_data,train_data[train_data.unit_ID == test_id]])
      train_data = train_data[train_data.unit_ID != test_id]

  ## create windowed dataset:
  basetable_x, basetable_y, train_UL = preprocess_train_windowed_UL(train_data,window_size=window_size)
  train_UL = train_UL.reshape((train_UL.shape[0],))

  #calculate cost of rul
  cost_RUL = acquisition_cost/train_UL 

  basetable_y=basetable_y.astype(np.float32)
  cost_RUL = cost_RUL.astype(np.float32)
  cost_RUL = cost_RUL.reshape((cost_RUL.shape[0],))

  if shared:
    cost_RUL= np.mean(cost_RUL)
  # same for test set:
  basetable_x_test, test_y, test_UL = preprocess_test_windowed_UL(test_data,window_size=window_size)
  test_UL = [tul.reshape((tul.shape[0],)) for tul in test_UL]
  cost_RUL_test = [(acquisition_cost/ul).astype(np.float32) for ul in test_UL] #calculate cost of rul
  test_y= [y.astype(np.float32) for y in test_y]

  #### Scale & right format for XGB
  scaler = StandardScaler()

  train_x, train_y = basetable_x,basetable_y
  train_x=scaler.fit_transform(train_x.reshape(-1,train_x.shape[-1])).reshape(train_x.shape)
  test_x=[scaler.transform(machine.reshape(-1,machine.shape[-1])).reshape(machine.shape) for machine in basetable_x_test]

  ## Right format for XGBOOST:
  train_x=train_x.reshape(train_x.shape[0],train_x.shape[1]*train_x.shape[2])
  test_x=[machine.reshape(machine.shape[0],machine.shape[1]*machine.shape[2]) for machine in test_x]

  ## Transform to natural logarithm (to enforce the labels to be strictly positive after e^(pred))
  train_y[train_y==0] = 1e-6
  # train_y=np.log(train_y)

  #DATM
  train_datm=xgb.DMatrix(train_x,label=train_y)
  test_x=[xgb.DMatrix(item) for item in test_x ]

  if not objective:
    #obj function, not specified, so choose prmcloss:
    objf = XGBObjectiveFunction(cost_reactive, cost_predictive, cost_RUL)
  else:
    objf = objective
  
  if type(objf) == str:
    bst = xgb.XGBRegressor(params, num_boost_round=epochs, obj=objf)
    bst.fit(train_datm)
  else:
    bst = xgb.train(params, train_datm, num_boost_round=epochs, obj=objf,evals=[(train_datm,'train')], custom_metric=objf.metric)

  preds_iter = unravel_labels_testset(predict_test_set(bst,test_x))
  trues_iter = unravel_labels_testset(test_y)
  cost_rul_iter = np.array([item[0] for item in cost_RUL_test])

  try: 
    with open(f'predictions/m{MODEL_NAME}.pkl','rb') as r:
      pk = pickle.load(r)
    preds = pk['preds']
    trues = pk['trues']
    cost_rul = pk['cost_rul']

    preds.append(preds_iter)
    trues.append(trues_iter)
    cost_rul.append(cost_rul_iter)
    print('read file & append')
  except:
    preds = [preds_iter]
    trues = [trues_iter]
    cost_rul = [cost_rul_iter]
    print('file did not exist yet')
  finally:
    with open(f'predictions/m{MODEL_NAME}.pkl','wb') as w:
      pickle.dump({
          'preds':preds,
          'trues':trues,
          'cost_rul':cost_rul,
          'cost_reactive':cost_reactive,
          'cost_predictive':cost_predictive
      },w)



# **Start simulation run:**


In [ ]:
def prep_train_val_set(train,validation,window_size=20,name='PRMC'):
    train_x, train_y, train_UL = preprocess_train_windowed_UL(train,window_size=window_size)
    ##Reshape to the right formats:
    train_UL = train_UL.reshape((train_UL.shape[0],))
    train_y=train_y.reshape((train_y.shape[0],))
    scaler = StandardScaler()
    train_x=scaler.fit_transform(train_x.reshape(-1,train_x.shape[-1])).reshape(train_x.shape)
    train_x=train_x.reshape(train_x.shape[0],train_x.shape[1]*train_x.shape[2])
    if name == 'PRMC':
        val_x, val_y, val_UL = preprocess_test_windowed_UL(validation,window_size=window_size)
        val_UL = [tul.reshape((tul.shape[0],)) for tul in val_UL]
        val_y= [y.astype(np.float32) for y in val_y]
        val_x=[scaler.transform(machine.reshape(-1,machine.shape[-1])).reshape(machine.shape) for machine in val_x]
        val_x=[machine.reshape(machine.shape[0],machine.shape[1]*machine.shape[2]) for machine in val_x]
    else:
        val_x, val_y, val_UL = preprocess_train_windowed_UL(validation,window_size=window_size)
        val_UL = val_UL.reshape((val_UL.shape[0],))
        val_y=val_y.reshape((val_y.shape[0],))
        val_x=scaler.transform(val_x.reshape(-1,val_x.shape[-1])).reshape(val_x.shape)
        val_x=val_x.reshape(val_x.shape[0],val_x.shape[1]*val_x.shape[2])




    return ((train_x, train_y, train_UL), (val_x, val_y, val_UL))



In [ ]:
from sklearn.metrics._scorer import _PredictScorer

class PRMCScorer(_PredictScorer):
    def __init__(self, cost_reactive, cost_predictive, cost_rul, tau=12):
        self.tau=tau
        self.cost_reactive=cost_reactive
        self.cost_predictive=cost_predictive
        self.cost_rul=cost_rul
    
    def __call__(self, estimator, X, y_true):
        preds = unravel_labels_testset(predict_test_set(estimator,X))
        trues = unravel_labels_testset(y_true)
        return np.min([np.sum(calculate_PRMC(preds,trues,self.tau,ti,self.cost_reactive,self.cost_predictive,self.cost_rul)) for ti in np.arange(0,preds.shape[-1])])

In [ ]:
train = pd.read_csv('data/gold/train.csv')
validation = pd.read_csv('data/gold/validation.csv')
acquisition_cost=50_000_000

# metric_name= 'neg_mean_squared_error'
metric_name= 'neg_mean_absolute_error'
# metric_name= 'PRMC'


# model_name = 'RF'
# model_name = 'XGBOOST'
# model_name = 'SVR'
model_name = 'LR'


if model_name == 'XGBOOST':
  model = xgb.XGBRegressor()
  params={
    'max_depth':np.arange(2,6),
    'n_estimators':np.arange(100,600,100),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [1, 0.9, 0.8, 0.7],
    'colsample_by_tree': [1, 0.9, 0.8, 0.7],
    'window_size':[20]
    }
elif model_name == 'SVR':
  model = LinearSVR()
  params={
      'epsilon':np.arange(6.4,6.7,0.1),
      'max_iter':[1000],
      'C':np.arange(17,24),
      'window_size':[40]
  }
elif model_name == 'RF':
  #For MAE use the XGBOOST implementation!
  model=XGBRFRegressor()
  params={
      'num_parallel_tree':[100],
      'eta':[0.1],
      'subsample':[0.9],
      'colsample_bynode':[0.9],
      'window_size':[40],
      'max_depth': [15],
      'objective': ['reg:absoluteerror'],
  }
elif model_name == 'LR':
  model=Ridge()
  params ={
      'alpha': np.arange(0,20,0.5),
      'max_iter':[100,150,200],
      'window_size':[40]
  }


#save the dataframes to a dict: (else you would need to make them everytime)
data=dict()
for ws in params['window_size']:
    data[str(ws)] = prep_train_val_set(train,validation,ws,name=metric_name)


num_combs = len(ParameterGrid(params))
print(f"Number of combinations: {num_combs}")
if metric_name == 'PRMC':
  best_eval = np.inf
  for iter,params in enumerate(ParameterGrid(params)): 
    print(f"{iter+1}/{num_combs}")
    (train,validation)=data[str(params['window_size'])]
    window_size = params['window_size']
    params.pop('window_size')
    model=model.__class__(**params)

    model.fit(train[0],train[1])
    cost_RUL_val = [(50_000_000/ul).astype(np.float32) for ul in validation[2]] #calculate cost of rul
    cost_RUL_val = np.array([item[0] for item in cost_RUL_val]) #keep one cost per machine
    prmc=PRMCScorer(50_000_000/3, 50_000_000/9, cost_RUL_val)
    new_eval = prmc(model,validation[0],validation[1])
    if new_eval < best_eval:
      params['window_size']=window_size
      best_params = params
      best_eval = new_eval
else:
  best_eval = -np.inf
  for iter,params in enumerate(ParameterGrid(params)):
    print(f"{iter+1}/{num_combs}")
    (train,validation)=data[str(params['window_size'])]
    window_size = params['window_size']
    params.pop('window_size')
    model=model.__class__(**params)
    model.fit(train[0],train[1])
    new_eval = _compute_score(model,validation[0],validation[1],scoring_metric=metric_name)
    if new_eval > best_eval:
      params['window_size']=window_size
      best_params = params
      best_eval = new_eval


print(f"#Best parameters: {best_params} Best Validation score: {abs(best_eval)} {metric_name}")







Number of combinations: 1
1/1
